In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker as tk
import scipy as sp

rPath = "../RESULTS/"
prTrain = rPath+"training/"

In [2]:
# It is important that these be arrays because they need to be
# in the same order as the parameters.h file
pops = ["V1L4E", "V1L4I", "V1L23E", "V1L23I", "V2L4E", "V2L4I", "V2L23E", "V2L23I", "LGN"]
dims = {"V1L4E":[23,4], "V1L4I":[23,1], "V1L23E":[17,7], "V1L23I":[17,2], "V2L4E":[13,10], "V2L4I":[13,3], "V2L23E":[10,16], "V2L23I":[10,5], "LGN":[32,2]}
RFsv = {"FF":{"V1L4":10, "V1L23":7, "V2L4":5, "V2L23":4}, "Lat":{"V1L4":11, "V1L23":9, "V2L4":7, "V2L23":5}, "FB":{"V1L4":7, "V1L23":8, "V2L4":4, "V2L23":0}}
syns = [ ["LGN", "V1L4E", "FF"], ["LGN", "V1L4I", "FF"], ["V1L4E", "V1L23E", "FF"], ["V1L4E", "V1L23I", "FF"], 
    ["V1L23E", "V2L4E", "FF"], ["V1L23E", "V2L4I", "FF"], ["V2L4E", "V2L23E", "FF"], ["V2L4E", "V2L23I", "FF"],
    ["V1L4E", "V1L4I", "Lat"], ["V1L4I", "V1L4E", "Lat"], ["V1L4I", "V1L4I", "Lat"], ["V1L23E", "V1L23I", "Lat"],
    ["V1L23I", "V1L23E", "Lat"], ["V1L23I", "V1L23I", "Lat"], ["V2L4E", "V2L4I", "Lat"], ["V2L4I", "V2L4E", "Lat"], 
    ["V2L4I", "V2L4I", "Lat"], ["V2L23E", "V2L23I", "Lat"], ["V2L23I", "V2L23E", "Lat"], ["V2L23I", "V2L23I", "Lat"], 
    ["V2L23E", "V1L23E", "FB"], ["V2L23E", "V1L23I", "FB"],  ["V1L23E", "V1L4I", "FB"], ["V2L23E", "V2L4I", "FB"], 
    ["V1L4I", "V1L23E", "FF"], ["V1L4I", "V1L23I", "FF"], ["V2L4I", "V2L23E", "FF"], ["V2L4I", "V2L23I", "FF"] ]
sNames = [ s[0]+"-"+s[1] for s in syns]

#  Training Results

## Weights

In [3]:
for s in syns:
    sName = s[0]+"-"+s[1]
    wCounts = []
    wBins = []
    with open(prTrain+sName+"dist.vec") as f:
        for i, r in enumerate(f):
            if (i%2 == 0): wCounts.append(np.fromstring(r, sep=" "))
            else: wBins.append(np.fromstring(r, sep=" "))
        fig, axs = plt.subplots(4,3, figsize=(20,10))
        toshow = [0, 1, 2, 5, 10, 20, 30, 40, 50, 100, 150, 200]
        form = tk.ScalarFormatter(useMathText=True)
        form.set_scientific(True)
        form.set_powerlimits((-1,1))
        for i, t in enumerate(toshow):
            axs.flatten()[i].bar(wBins[t][:-1], wCounts[t], width=wBins[t][1]*.8, align='center')
            axs.flatten()[i].set_title(str(t*1000) + " patches")
            axs.flatten()[i].yaxis.set_major_formatter(form)
        fig.suptitle(sName)
        fig.tight_layout()
        fig.savefig(rPath+"training_sum/"+sName+"_wDists.png")
        plt.close(fig)

In [4]:
wAvg = pd.read_csv(prTrain+"avgWeights.tsv", sep="\t")
for p in pops:
    cols = [c for c in wAvg.columns if p+"-" in c]
    wAvg[cols].plot(figsize=(10,7))
    plt.title("Source: "+p)
    plt.legend(title="Target", labels=[c.split("-")[1] for c in cols])
    plt.savefig(rPath+"training_sum/"+p+"-wAvg.png")
    plt.close()


## Rates

In [41]:
for p in pops:
    rCounts = []
    rBins = []
    with open(prTrain+p+"dist.vec") as f:
        for i, r in enumerate(f):
            if (i%2 == 0): rCounts.append(np.fromstring(r, sep=" "))
            else: rBins.append(np.fromstring(r, sep=" "))
        fig, axs = plt.subplots(4,3, figsize=(20,10))
        toshow = [1, 2, 5, 10, 20, 30, 40, 50, 80, 100, 150, 200]
        form = tk.ScalarFormatter(useMathText=True)
        form.set_scientific(True)
        form.set_powerlimits((-1,1))
        for i, t in enumerate(toshow):
            axs.flatten()[i].bar(wBins[t][:-1], rCounts[t], width=rBins[t][1]*.8, align='center')
            axs.flatten()[i].set_title(str(t*1000) + " patches")
            axs.flatten()[i].yaxis.set_major_formatter(form)
        fig.suptitle(p)
        fig.tight_layout()
        fig.savefig(rPath+"training_sum/"+p+"_rDists.png")
        plt.close(fig)

In [6]:
rAvg = pd.read_csv(prTrain+"avgRates.tsv", sep="\t")
rAvg[pops].apply(lambda x : sp.signal.savgol_filter(x,20,2)).plot()
plt.title("Smoothed rates")
plt.savefig(rPath+"training_sum/avgRates.png")
plt.close()

## Receptive Fields

In [ ]:
from scipy.signal import gaussian
gaussian

for s in syns[:2]:
    sName = s[0]+"-"+s[1]
    mat = np.loadtxt(prTrain+sName+"weights.vec")
    mask = np.loadtxt("architecture/matrices/"+sName)
    fE, axE = plt.subplots(10, 10, figsize = (10,10))
    fE.suptitle(sName+" On", y=.92)    
    fI, axI = plt.subplots(10, 10, figsize = (10,10))
    fI.suptitle(sName+" Off", y=.92)
    fRF, axRF = plt.subplots(10, 10, figsize = (10,10))
    fRF.suptitle(sName+" Receptive Fields", y=.92)
    for i in range(100):
        rf = mat[:,100+i][mask[:,100+i]!=0.].reshape(10,10,2)
        exc = rf[:,:,0]/rf[:,:,0].max()
        inh = rf[:,:,1]/rf[:,:,1].max()
        axE[i//10][i%10].matshow(exc, cmap=plt.cm.Blues)
        axE[i//10][i%10].set_xticks([],[])
        axE[i//10][i%10].set_yticks([],[])
        axI[i//10][i%10].matshow(inh, cmap=plt.cm.Reds)
        axI[i//10][i%10].set_xticks([],[])
        axI[i//10][i%10].set_yticks([],[])
        axRF[i//10][i%10].matshow(inh-exc, cmap=plt.cm.bwr)
        axRF[i//10][i%10].set_xticks([],[])
        axRF[i//10][i%10].set_yticks([],[])
    fE.savefig(rPath+"training_sum/"+sName+"On")
    fI.savefig(rPath+"training_sum/"+sName+"Off")
    fRF.savefig(rPath+"training_sum/"+sName+"RF")